In [1]:
import argparse
import torch
import pandas as pd
from tqdm import tqdm
import os
import time
from concurrent.futures import ThreadPoolExecutor
from vllm import LLM, SamplingParams

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
model_id = "google/gemma-2-9b-it"
llm = LLM(model=model_id, gpu_memory_utilization=0.7, dtype=torch.bfloat16, device='cuda')

INFO 05-18 03:28:13 __init__.py:207] Automatically detected platform cuda.


INFO 05-18 03:28:22 config.py:549] This model supports multiple tasks: {'reward', 'generate', 'classify', 'embed', 'score'}. Defaulting to 'generate'.


INFO 05-18 03:28:22 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='google/gemma-2-9b-it', speculative_config=None, tokenizer='google/gemma-2-9b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=google/gemma-2-9b-it, num_scheduler_steps=1, multi_step_stream_outputs=True, enable_prefix_caching=False, chunked_prefill_enabled=False, use_async_output_proc=True, disable_mm_preprocessor_cache=

INFO 05-18 03:28:25 cuda.py:229] Using Flash Attention backend.


INFO 05-18 03:28:26 model_runner.py:1110] Starting to load model google/gemma-2-9b-it...


...

Capturing CUDA graph shapes:  97%|█████████▋| 34/35 [00:15<00:00,  2.36it/s]

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:15<00:00,  2.17it/s]

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:15<00:00,  2.19it/s]

INFO 05-18 03:28:52 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.41 GiB


INFO 05-18 03:28:52 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 19.57 seconds


In [4]:
folder = 'gemma2_9b'

In [5]:
def output_gen_batch(prompts,max_token=4096):
    try:
        sampling_params = SamplingParams(temperature=0, top_p=0.8, max_tokens=max_token, top_k=10, detokenize=True)
        outputs = llm.chat(messages=prompts,sampling_params=sampling_params)
        # outputs = llm.generate(prompts,sampling_params=sampling_params)
        # print(outputs)
        return [o.outputs[0].text for o in outputs]
    except Exception as e:
        return [f"Error: {e}" for _ in prompts]

## English

In [6]:
def pipe_line_batch_chat(prompts1,prompts2 ,tgt_lang):
    chat_histories = []

    system_prompt = """"""

    # Step 1: Generate initial completions
    initial_chats = [{"role": "user", "content": f"{prompt}\n{system_prompt}"} for prompt in prompts1]
    initial_outputs = output_gen_batch([[chat] for chat in initial_chats])  # Assume it expects batch of chat lists

    # Step 2: Build full conversation with initial response
    for prompt1,prompt2, initial in zip(prompts1, prompts2, initial_outputs):
        chat = [
            {"role": "user", "content": f"{prompt1}\n{system_prompt}"},
            {"role": "assistant", "content": initial},
            {"role": "user", "content": f"{prompt2}\n{system_prompt}"}
        ]
        chat_histories.append(chat)

    # Step 3: Generate final outputs
    final_outputs = output_gen_batch(chat_histories)
    return initial_outputs,final_outputs


In [7]:
language = 'english'

In [8]:
def process_batch(df, tgt_lang,batch_size=16):
    batch_responses = []

    for idx in range(0, len(df), batch_size):
        batch_df = df.iloc[idx:idx + batch_size]
        prompts1 = batch_df[f'{language}_history_prompt_1'].tolist()
        prompts2 = batch_df[f'{language}_history_prompt_2'].tolist()

        try:
            start_time = time.time()
            initial_outputs,output1_list = pipe_line_batch_chat(prompts1,prompts2, tgt_lang=tgt_lang)
            end_time = time.time()

            print(f"Processed batch {idx}-{idx+len(batch_df)} in {end_time - start_time:.2f} seconds")

            for i, row in batch_df.iterrows():
                row_dict = row.to_dict()
                row_dict['initial_response'] = initial_outputs[i - idx]  # Align index within batch
                row_dict['final_response'] = output1_list[i - idx]  # Align index within batch
                batch_responses.append(row_dict)
        except Exception as e:
            for _, row in batch_df.iterrows():
                row_dict = row.to_dict()
                row_dict['initial_response'] = f"Error: {e}"
                row_dict['final_response'] = f"Error: {e}"
                batch_responses.append(row_dict)

    return batch_responses


In [9]:
output_folder = f'/home/nihar.sahoo/Jailbreak/Aakash/New/jailbreak_responses/history/{folder}/english'

In [10]:
def process_csv_file():
    df = pd.read_csv(f"/home/nihar.sahoo/Jailbreak/Aakash/New/history/processed/history_{language}.csv")
    #df = df[:10]
    tgt_lang = language
    all_responses = process_batch(df, tgt_lang,batch_size=32)
    output_file_path = os.path.join(output_folder, "eng.parquet")
    responses_df = pd.DataFrame(all_responses)
    responses_df.to_parquet(output_file_path, index=False)
    print(f"Processed and saved: {output_file_path}")


In [11]:
process_csv_file()

INFO 05-18 03:28:53 chat_utils.py:332] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   3%|▎         | 1/32 [00:01<01:00,  1.94s/it, est. speed input: 16.46 toks/s, output: 83.32 toks/s]

Processed prompts:   6%|▋         | 2/32 [00:02<00:37,  1.26s/it, est. speed input: 24.56 toks/s, output: 143.30 toks/s]

Processed prompts:   9%|▉         | 3/32 [00:02<00:22,  1.27it/s, est. speed input: 34.18 toks/s, output: 216.22 toks/s]

...

Processed prompts:  25%|██▌       | 1/4 [00:03<00:11,  3.68s/it, est. speed input: 137.13 toks/s, output: 99.04 toks/s]

Processed prompts:  50%|█████     | 2/4 [00:05<00:04,  2.43s/it, est. speed input: 198.40 toks/s, output: 169.00 toks/s]

Processed prompts: 100%|██████████| 4/4 [00:05<00:00,  1.02s/it, est. speed input: 372.64 toks/s, output: 351.78 toks/s]

Processed prompts: 100%|██████████| 4/4 [00:05<00:00,  1.40s/it, est. speed input: 372.64 toks/s, output: 351.78 toks/s]

Processed batch 1664-1668 in 10.82 seconds
Processed and saved: /home/nihar.sahoo/Jailbreak/Aakash/New/jailbreak_responses/history/gemma2_9b/english/eng.parquet


## Hindi

In [12]:
language = 'hindi'

In [13]:
def pipe_line_batch_chat(prompts1,prompts2 ,tgt_lang):
    chat_histories = []

    system_prompt = """"""

    # Step 1: Generate initial completions
    initial_chats = [{"role": "user", "content": f"{prompt}\n{system_prompt}"} for prompt in prompts1]
    initial_outputs = output_gen_batch([[chat] for chat in initial_chats])  # Assume it expects batch of chat lists

    # Step 2: Build full conversation with initial response
    for prompt1,prompt2, initial in zip(prompts1, prompts2, initial_outputs):
        chat = [
            {"role": "user", "content": f"{prompt1}\n{system_prompt}"},
            {"role": "assistant", "content": initial},
            {"role": "user", "content": f"{prompt2}\n{system_prompt}"}
        ]
        chat_histories.append(chat)

    # Step 3: Generate final outputs
    final_outputs = output_gen_batch(chat_histories)
    return initial_outputs,final_outputs


In [14]:
def process_batch(df, tgt_lang,batch_size=16):
    batch_responses = []

    for idx in range(0, len(df), batch_size):
        batch_df = df.iloc[idx:idx + batch_size]
        prompts1 = batch_df[f'{language}_history_prompt_1'].tolist()
        prompts2 = batch_df[f'{language}_history_prompt_2'].tolist()

        try:
            start_time = time.time()
            initial_outputs,output1_list = pipe_line_batch_chat(prompts1,prompts2, tgt_lang=tgt_lang)
            end_time = time.time()

            print(f"Processed batch {idx}-{idx+len(batch_df)} in {end_time - start_time:.2f} seconds")

            for i, row in batch_df.iterrows():
                row_dict = row.to_dict()
                row_dict['initial_response'] = initial_outputs[i - idx]  # Align index within batch
                row_dict['final_response'] = output1_list[i - idx]  # Align index within batch
                batch_responses.append(row_dict)
        except Exception as e:
            for _, row in batch_df.iterrows():
                row_dict = row.to_dict()
                row_dict['initial_response'] = f"Error: {e}"
                row_dict['final_response'] = f"Error: {e}"
                batch_responses.append(row_dict)

    return batch_responses


In [15]:
output_folder = f'/home/nihar.sahoo/Jailbreak/Aakash/New/jailbreak_responses/history/{folder}/{language}'

In [16]:
def process_csv_file():
    df = pd.read_csv(f"/home/nihar.sahoo/Jailbreak/Aakash/New/history/processed/history_{language}.csv")
    #df = df[:10]
    tgt_lang = language
    all_responses = process_batch(df, tgt_lang,batch_size=32)
    output_file_path = os.path.join(output_folder, "hin_hin.parquet")
    responses_df = pd.DataFrame(all_responses)
    responses_df.to_parquet(output_file_path, index=False)
    print(f"Processed and saved: {output_file_path}")


In [17]:
process_csv_file()

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   3%|▎         | 1/32 [00:01<00:53,  1.72s/it, est. speed input: 33.74 toks/s, output: 82.60 toks/s]

Processed prompts:   6%|▋         | 2/32 [00:02<00:37,  1.25s/it, est. speed input: 45.32 toks/s, output: 137.47 toks/s]

Processed prompts:   9%|▉         | 3/32 [00:04<00:44,  1.53s/it, est. speed input: 39.79 toks/s, output: 164.51 toks/s]

Processed prompts:  19%|█▉        | 6/32 [00:04<00:16,  1.61it/s, est. speed input: 72.56 toks/s, output: 384.49 toks/s]

...

Processed prompts:  50%|█████     | 2/4 [00:04<00:04,  2.04s/it, est. speed input: 113.32 toks/s, output: 177.11 toks/s]

Processed prompts:  75%|███████▌  | 3/4 [00:05<00:01,  1.39s/it, est. speed input: 177.14 toks/s, output: 254.99 toks/s]

Processed prompts: 100%|██████████| 4/4 [00:07<00:00,  1.85s/it, est. speed input: 225.73 toks/s, output: 271.81 toks/s]

Processed prompts: 100%|██████████| 4/4 [00:07<00:00,  1.93s/it, est. speed input: 225.73 toks/s, output: 271.81 toks/s]

Processed batch 1664-1668 in 14.38 seconds
Processed and saved: /home/nihar.sahoo/Jailbreak/Aakash/New/jailbreak_responses/history/gemma2_9b/hindi/hin_hin.parquet


## telugu

In [18]:
language = 'telugu'

In [19]:
def pipe_line_batch_chat(prompts1,prompts2 ,tgt_lang):
    chat_histories = []

    system_prompt = """"""

    # Step 1: Generate initial completions
    initial_chats = [{"role": "user", "content": f"{prompt}\n{system_prompt}"} for prompt in prompts1]
    initial_outputs = output_gen_batch([[chat] for chat in initial_chats])  # Assume it expects batch of chat lists

    # Step 2: Build full conversation with initial response
    for prompt1,prompt2, initial in zip(prompts1, prompts2, initial_outputs):
        chat = [
            {"role": "user", "content": f"{prompt1}\n{system_prompt}"},
            {"role": "assistant", "content": initial},
            {"role": "user", "content": f"{prompt2}\n{system_prompt}"}
        ]
        chat_histories.append(chat)

    # Step 3: Generate final outputs
    final_outputs = output_gen_batch(chat_histories)
    return initial_outputs,final_outputs


In [20]:
def process_batch(df, tgt_lang,batch_size=16):
    batch_responses = []

    for idx in range(0, len(df), batch_size):
        batch_df = df.iloc[idx:idx + batch_size]
        prompts1 = batch_df[f'{language}_history_prompt_1'].tolist()
        prompts2 = batch_df[f'{language}_history_prompt_2'].tolist()

        try:
            start_time = time.time()
            initial_outputs,output1_list = pipe_line_batch_chat(prompts1,prompts2, tgt_lang=tgt_lang)
            end_time = time.time()

            print(f"Processed batch {idx}-{idx+len(batch_df)} in {end_time - start_time:.2f} seconds")

            for i, row in batch_df.iterrows():
                row_dict = row.to_dict()
                row_dict['initial_response'] = initial_outputs[i - idx]  # Align index within batch
                row_dict['final_response'] = output1_list[i - idx]  # Align index within batch
                batch_responses.append(row_dict)
        except Exception as e:
            for _, row in batch_df.iterrows():
                row_dict = row.to_dict()
                row_dict['initial_response'] = f"Error: {e}"
                row_dict['final_response'] = f"Error: {e}"
                batch_responses.append(row_dict)

    return batch_responses


In [21]:
output_folder = f'/home/nihar.sahoo/Jailbreak/Aakash/New/jailbreak_responses/history/{folder}/{language}'

In [22]:
def process_csv_file():
    df = pd.read_csv(f"/home/nihar.sahoo/Jailbreak/Aakash/New/history/processed/history_{language}.csv")
    #df = df[:10]
    tgt_lang = language
    all_responses = process_batch(df, tgt_lang,batch_size=32)
    output_file_path = os.path.join(output_folder, "tel_tel.parquet")
    responses_df = pd.DataFrame(all_responses)
    responses_df.to_parquet(output_file_path, index=False)
    print(f"Processed and saved: {output_file_path}")


In [23]:
process_csv_file()

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   3%|▎         | 1/32 [00:02<01:12,  2.33s/it, est. speed input: 30.89 toks/s, output: 84.09 toks/s]

Processed prompts:   6%|▋         | 2/32 [00:02<00:30,  1.02s/it, est. speed input: 59.05 toks/s, output: 164.43 toks/s]

Processed prompts:   9%|▉         | 3/32 [00:04<00:48,  1.68s/it, est. speed input: 43.28 toks/s, output: 165.17 toks/s]

Processed prompts:  12%|█▎        | 4/32 [00:05<00:36,  1.31s/it, est. speed input: 48.26 toks/s, output: 226.40 toks/s]

...

Processed prompts:  50%|█████     | 2/4 [00:10<00:11,  5.60s/it, est. speed input: 167.92 toks/s, output: 120.27 toks/s]

Processed prompts:  75%|███████▌  | 3/4 [00:11<00:03,  3.51s/it, est. speed input: 241.79 toks/s, output: 203.88 toks/s]

Processed prompts: 100%|██████████| 4/4 [00:12<00:00,  2.63s/it, est. speed input: 330.37 toks/s, output: 279.07 toks/s]

Processed prompts: 100%|██████████| 4/4 [00:12<00:00,  3.18s/it, est. speed input: 330.37 toks/s, output: 279.07 toks/s]

Processed batch 1664-1668 in 24.88 seconds
Processed and saved: /home/nihar.sahoo/Jailbreak/Aakash/New/jailbreak_responses/history/gemma2_9b/telugu/tel_tel.parquet


## marathi

In [24]:
language = 'marathi'

In [25]:
def pipe_line_batch_chat(prompts1,prompts2 ,tgt_lang):
    chat_histories = []

    system_prompt = """"""

    # Step 1: Generate initial completions
    initial_chats = [{"role": "user", "content": f"{prompt}\n{system_prompt}"} for prompt in prompts1]
    initial_outputs = output_gen_batch([[chat] for chat in initial_chats])  # Assume it expects batch of chat lists

    # Step 2: Build full conversation with initial response
    for prompt1,prompt2, initial in zip(prompts1, prompts2, initial_outputs):
        chat = [
            {"role": "user", "content": f"{prompt1}\n{system_prompt}"},
            {"role": "assistant", "content": initial},
            {"role": "user", "content": f"{prompt2}\n{system_prompt}"}
        ]
        chat_histories.append(chat)

    # Step 3: Generate final outputs
    final_outputs = output_gen_batch(chat_histories)
    return initial_outputs,final_outputs


In [26]:
def process_batch(df, tgt_lang,batch_size=16):
    batch_responses = []

    for idx in range(0, len(df), batch_size):
        batch_df = df.iloc[idx:idx + batch_size]
        prompts1 = batch_df[f'{language}_history_prompt_1'].tolist()
        prompts2 = batch_df[f'{language}_history_prompt_2'].tolist()

        try:
            start_time = time.time()
            initial_outputs,output1_list = pipe_line_batch_chat(prompts1,prompts2, tgt_lang=tgt_lang)
            end_time = time.time()

            print(f"Processed batch {idx}-{idx+len(batch_df)} in {end_time - start_time:.2f} seconds")

            for i, row in batch_df.iterrows():
                row_dict = row.to_dict()
                row_dict['initial_response'] = initial_outputs[i - idx]  # Align index within batch
                row_dict['final_response'] = output1_list[i - idx]  # Align index within batch
                batch_responses.append(row_dict)
        except Exception as e:
            for _, row in batch_df.iterrows():
                row_dict = row.to_dict()
                row_dict['initial_response'] = f"Error: {e}"
                row_dict['final_response'] = f"Error: {e}"
                batch_responses.append(row_dict)

    return batch_responses


In [27]:
output_folder = f'/home/nihar.sahoo/Jailbreak/Aakash/New/jailbreak_responses/history/{folder}/{language}'

In [28]:
def process_csv_file():
    df = pd.read_csv(f"/home/nihar.sahoo/Jailbreak/Aakash/New/history/processed/history_{language}.csv")
    #df = df[:10]
    tgt_lang = language
    all_responses = process_batch(df, tgt_lang,batch_size=32)
    output_file_path = os.path.join(output_folder, "mar_mar.parquet")
    responses_df = pd.DataFrame(all_responses)
    responses_df.to_parquet(output_file_path, index=False)
    print(f"Processed and saved: {output_file_path}")


In [29]:
process_csv_file()

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   3%|▎         | 1/32 [00:02<01:07,  2.19s/it, est. speed input: 32.83 toks/s, output: 85.28 toks/s]

Processed prompts:   6%|▋         | 2/32 [00:02<00:36,  1.22s/it, est. speed input: 54.13 toks/s, output: 153.60 toks/s]

Processed prompts:   9%|▉         | 3/32 [00:02<00:20,  1.40it/s, est. speed input: 71.42 toks/s, output: 232.91 toks/s]

Processed prompts:  12%|█▎        | 4/32 [00:05<00:38,  1.37s/it, est. speed input: 50.94 toks/s, output: 210.65 toks/s]

...

Processed prompts:  25%|██▌       | 1/4 [00:02<00:07,  2.54s/it, est. speed input: 115.57 toks/s, output: 93.87 toks/s]

Processed prompts:  50%|█████     | 2/4 [00:08<00:09,  4.78s/it, est. speed input: 145.46 toks/s, output: 122.73 toks/s]

Processed prompts:  75%|███████▌  | 3/4 [00:12<00:04,  4.34s/it, est. speed input: 184.25 toks/s, output: 183.39 toks/s]

Processed prompts: 100%|██████████| 4/4 [00:12<00:00,  3.18s/it, est. speed input: 274.37 toks/s, output: 280.26 toks/s]

Processed batch 1664-1668 in 22.21 seconds
Processed and saved: /home/nihar.sahoo/Jailbreak/Aakash/New/jailbreak_responses/history/gemma2_9b/marathi/mar_mar.parquet


## bengali

In [30]:
language = 'bengali'

In [31]:
def pipe_line_batch_chat(prompts1,prompts2 ,tgt_lang):
    chat_histories = []

    system_prompt = """"""

    # Step 1: Generate initial completions
    initial_chats = [{"role": "user", "content": f"{prompt}\n{system_prompt}"} for prompt in prompts1]
    initial_outputs = output_gen_batch([[chat] for chat in initial_chats])  # Assume it expects batch of chat lists

    # Step 2: Build full conversation with initial response
    for prompt1,prompt2, initial in zip(prompts1, prompts2, initial_outputs):
        chat = [
            {"role": "user", "content": f"{prompt1}\n{system_prompt}"},
            {"role": "assistant", "content": initial},
            {"role": "user", "content": f"{prompt2}\n{system_prompt}"}
        ]
        chat_histories.append(chat)

    # Step 3: Generate final outputs
    final_outputs = output_gen_batch(chat_histories)
    return initial_outputs,final_outputs


In [32]:
def process_batch(df, tgt_lang,batch_size=16):
    batch_responses = []

    for idx in range(0, len(df), batch_size):
        batch_df = df.iloc[idx:idx + batch_size]
        prompts1 = batch_df[f'{language}_history_prompt_1'].tolist()
        prompts2 = batch_df[f'{language}_history_prompt_2'].tolist()

        try:
            start_time = time.time()
            initial_outputs,output1_list = pipe_line_batch_chat(prompts1,prompts2, tgt_lang=tgt_lang)
            end_time = time.time()

            print(f"Processed batch {idx}-{idx+len(batch_df)} in {end_time - start_time:.2f} seconds")

            for i, row in batch_df.iterrows():
                row_dict = row.to_dict()
                row_dict['initial_response'] = initial_outputs[i - idx]  # Align index within batch
                row_dict['final_response'] = output1_list[i - idx]  # Align index within batch
                batch_responses.append(row_dict)
        except Exception as e:
            for _, row in batch_df.iterrows():
                row_dict = row.to_dict()
                row_dict['initial_response'] = f"Error: {e}"
                row_dict['final_response'] = f"Error: {e}"
                batch_responses.append(row_dict)

    return batch_responses


In [33]:
output_folder = f'/home/nihar.sahoo/Jailbreak/Aakash/New/jailbreak_responses/history/{folder}/{language}'

In [34]:
def process_csv_file():
    df = pd.read_csv(f"/home/nihar.sahoo/Jailbreak/Aakash/New/history/processed/history_{language}.csv")
    #df = df[:10]
    tgt_lang = language
    all_responses = process_batch(df, tgt_lang,batch_size=32)
    output_file_path = os.path.join(output_folder, "beng_beng.parquet")
    responses_df = pd.DataFrame(all_responses)
    responses_df.to_parquet(output_file_path, index=False)
    print(f"Processed and saved: {output_file_path}")


In [35]:
process_csv_file()

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   3%|▎         | 1/32 [00:03<01:33,  3.02s/it, est. speed input: 29.52 toks/s, output: 84.57 toks/s]

Processed prompts:   6%|▋         | 2/32 [00:03<00:39,  1.33s/it, est. speed input: 52.19 toks/s, output: 165.11 toks/s]

Processed prompts:   9%|▉         | 3/32 [00:04<00:35,  1.22s/it, est. speed input: 56.55 toks/s, output: 206.87 toks/s]

Processed prompts:  12%|█▎        | 4/32 [00:04<00:28,  1.01s/it, est. speed input: 60.03 toks/s, output: 260.92 toks/s]

...

Processed prompts:  50%|█████     | 2/4 [00:05<00:05,  2.79s/it, est. speed input: 178.40 toks/s, output: 159.65 toks/s]

Processed prompts:  75%|███████▌  | 3/4 [00:07<00:02,  2.26s/it, est. speed input: 277.08 toks/s, output: 222.51 toks/s]

Processed prompts: 100%|██████████| 4/4 [00:10<00:00,  2.60s/it, est. speed input: 264.66 toks/s, output: 256.89 toks/s]

Processed prompts: 100%|██████████| 4/4 [00:10<00:00,  2.67s/it, est. speed input: 264.66 toks/s, output: 256.89 toks/s]

Processed batch 1664-1668 in 18.79 seconds
Processed and saved: /home/nihar.sahoo/Jailbreak/Aakash/New/jailbreak_responses/history/gemma2_9b/bengali/beng_beng.parquet
